In [1]:
import numpy as np
from Chempy.parameter import ModelParameters
import importlib
import fileinput
import sys   
from Chempy.wrapper import multi_star_optimization
from Chempy.plot_mcmc import restructure_chain
from Chempy.cem_function import posterior_function_returning_predictions

In [2]:
## Code to rewrite parameter file for each element in turn, so as to run MCMC for 21/22 elements only
# This is definitely not a good implementation (involves rewriting entire parameter file),
# But other steps are far slower

# Starting elements (copied from original parameter file)
elements_to_trace = ['Al', 'Ar', 'B', 'Be', 'C', 'Ca', 'Cl', 'Co', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'Ge', 'H', 'He', 'K', 'Li', 'Mg', 'Mn', 'N', 'Na', 'Ne', 'Ni', 'O', 'P', 'S', 'Sc', 'Si', 'Ti', 'V', 'Zn']
orig = "\telements_to_trace = "+str(elements_to_trace) # Original element string

# Calculate required Chempy elements
sol_dat = np.load("Chempy/input/stars/Proto-sun.npy")
el_names = []
for el in elements_to_trace:
    if el in sol_dat.dtype.names:
        el_names.append(el)
np.save("Hogg/elements.npy",np.array(el_names))
        
# Create new parameter names
newstr = []
for i,el in enumerate(el_names):
    if el !='Zn':
        newstr.append(orig.replace("'"+str(el)+"', ",""))
    else:
        newstr.append(orig.replace("'"+str(el)+"', ",""))
        
        
for i in range(2): #len(el_names)): # Iterate over removed element
    for line in fileinput.input("Chempy/parameter.py", inplace=True):
        if "\telements_to_trace" in line:
            print(newstr[i])
        else:
            print(line,end='')
    del sys.modules['Chempy.parameter']
    from Chempy.parameter import ModelParameters
    a = ModelParameters()
    ##############
    # MCMC using 21 elements only goes here
    print('Running MCMC iteration %d of %d' %(i+1,len(el_names)))
    multi_star_optimization()
    restructure_chain('mcmc/')
    positions = np.load('mcmc/posteriorPDF.npy')
    median = []
    up = []
    low = []
    for j in range(len(a.p0)):
        median.append(np.percentile(positions[:,j],50))
        low.append(np.percentile(positions[:,j],15.865))
        up.append(np.percentile(positions[:,j],100-15.865))
    np.save('Hogg/median%d.npy' %(i),np.array(median))
    np.save('Hogg/low%d.npy' %(i),np.array(low))
    np.save('Hogg/up%d.npy' %(i),np.array(up))
    ##############
    for line in fileinput.input("Chempy/parameter.py", inplace=True):
        if "\telements_to_trace" in line:
            print(orig)
        else:
            print(line,end='')
    del sys.modules['Chempy.parameter']
    from Chempy.parameter import ModelParameters
    a = ModelParameters()
    ##############
    # Code needing all 22 elements goes here
    param = median
    abundances,names = posterior_function_returning_predictions((param,a))
    for n,name in enumerate(names):
        if name == el_names[i]:
            required_abundance = abundances[n]
    np.save('Hogg/abundance%d.npy' %(i),required_abundance)

Running MCMC iteration 1 of 22
['Proto-sun']
first minimization for each star separately took:  1 seconds
[0.21641857172614709]
step 1 global minimization took: 56 seconds
step 1 local minimization took:  2 seconds
[0.21641857172614709, 0.59308274682243445]
step 2 global minimization took: 46 seconds
step 2 local minimization took:  1 seconds
[0.21641857172614709, 0.59308274682243445, 0.75645723661465536]
step 3 global minimization took: 46 seconds
step 3 local minimization took:  1 seconds
[0.21641857172614709, 0.59308274682243445, 0.75645723661465536, 0.80466109030213384]
step 4 global minimization took: 44 seconds
step 4 local minimization took:  1 seconds
[array([[-2.43245299, -2.8151126 , -0.73360428, -0.30470884,  0.57884549,
         0.53001605]]), 'initial minimization', (array([-2.37800746, -2.85745152, -0.76746838]), 0.21641857172614709), 'step 1 global minimization', array([[-2.37800746, -2.85745152, -0.76746838, -0.30132319,  0.5820613 ,
         0.48584804]]), 'step 1 loca

Traceback (most recent call last):
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 881, in posterior_function_many_stars
    posterior, blobs = posterior_function_many_stars_real(changing_parameter,error_list,elements)
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 919, in posterior_function_many_stars_real
    abundance_list,element_list = posterior_function_returning_predictions(args)
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 481, in posterior_function_returning_predictions
    posterior, abundance_list, element_list = posterior_function_predictions(changing_parameter,a)
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 491, in posterior_function_predictions
    a = extract_parameters_and_priors(changing_parameter, a)
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 716, in extract_parameters_and_priors
    assert False, '%s lower border is violated' %(name)
AssertionError: log10_sfr_scale lower border is violated


2.61487443274 0.341640355049
calculation so far took 253.6656937599182  seconds
step  40 of  1000
2.61487443274 0.141665995113
calculation so far took 259.97989654541016  seconds
step  41 of  1000
2.61487443274 0.219993738137
calculation so far took 266.1547691822052  seconds
step  42 of  1000
2.61487443274 0.383851532751
calculation so far took 271.483736038208  seconds
step  43 of  1000
2.61487443274 0.267613573525
calculation so far took 277.3230149745941  seconds
step  44 of  1000
2.61487443274 0.135959373043
calculation so far took 282.9647946357727  seconds
step  45 of  1000
2.61487443274 0.102611589281
calculation so far took 289.53657722473145  seconds
step  46 of  1000
2.61487443274 0.0173720623981
calculation so far took 295.31171798706055  seconds
step  47 of  1000
2.61487443274 0.170607004409
calculation so far took 301.1157422065735  seconds
step  48 of  1000


Traceback (most recent call last):
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 881, in posterior_function_many_stars
    posterior, blobs = posterior_function_many_stars_real(changing_parameter,error_list,elements)
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 919, in posterior_function_many_stars_real
    abundance_list,element_list = posterior_function_returning_predictions(args)
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 481, in posterior_function_returning_predictions
    posterior, abundance_list, element_list = posterior_function_predictions(changing_parameter,a)
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 491, in posterior_function_predictions
    a = extract_parameters_and_priors(changing_parameter, a)
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 716, in extract_parameters_and_priors
    assert False, '%s lower border is violated' %(name)
AssertionError: log10_sfr_scale lower border is violated


2.61487443274 0.251089979458
calculation so far took 306.4549388885498  seconds
step  49 of  1000
2.61487443274 0.097550060658
calculation so far took 312.0071370601654  seconds
step  50 of  1000
2.61487443274 0.0766198639257
calculation so far took 318.0062735080719  seconds
step  51 of  1000


...

Traceback (most recent call last):
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 881, in posterior_function_many_stars
    posterior, blobs = posterior_function_many_stars_real(changing_parameter,error_list,elements)
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 919, in posterior_function_many_stars_real
    abundance_list,element_list = posterior_function_returning_predictions(args)
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 481, in posterior_function_returning_predictions
    posterior, abundance_list, element_list = posterior_function_predictions(changing_parameter,a)
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 491, in posterior_function_predictions
    a = extract_parameters_and_priors(changing_parameter, a)
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 716, in extract_parameters_and_priors
    assert False, '%s lower border is violated' %(name)
AssertionError: log10_sfr_scale lower border is violated
Traceback (

2.61487443274 -0.29816927795
posterior -1, -100, -200 -0.29816927795 -0.239874939023 0.0554300692912
posterior 0, 100, 200 2.61487443274 0.0328071444165 -0.445764375137
calculation so far took 1787.9105205535889  seconds
step  302 of  1000


Traceback (most recent call last):
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 881, in posterior_function_many_stars
    posterior, blobs = posterior_function_many_stars_real(changing_parameter,error_list,elements)
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 919, in posterior_function_many_stars_real
    abundance_list,element_list = posterior_function_returning_predictions(args)
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 481, in posterior_function_returning_predictions
    posterior, abundance_list, element_list = posterior_function_predictions(changing_parameter,a)
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 491, in posterior_function_predictions
    a = extract_parameters_and_priors(changing_parameter, a)
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 718, in extract_parameters_and_priors
    assert False, '%s upper border is violated' %(name)
AssertionError: log10_sfr_scale upper border is violated


2.61487443274 -0.224750087713
posterior -1, -100, -200 -0.224750087713 -0.247640891152 0.0453927674098
posterior 0, 100, 200 2.61487443274 0.0328071444165 -0.445764375137
calculation so far took 1793.5763261318207  seconds
blob shape =  (64, 303, 21, 1) probably some runs did not return results and were stored anyway.
The chain has a length of 303 iterations, each iteration having 64 evaluations/walkers
Mean posteriors at the beginning and the end of the chain:
-0.224750087713 2.61487443274
Mean posteriors after the burn-in tail is cut out:
-0.224750087713 -0.00704753564691
We are left with a sample of 4992 posterior evaluations from the converged MCMC chain
We have 4992 iterations good enough posterior, their posteriors range from
2.90685328964 -10.6821308889
Highest posterior was obtained at parameters:  [[-2.32938599 -2.83787132 -0.90304963 -0.19666093  0.5879297   0.48285063]]
Number of unique posterior values:  2632
Inferred marginalized parameter distributions are:
0 -2.315690837

KeyboardInterrupt: 

In [6]:
np.load('Hogg/elements.npy')

array(['Al', 'Ar', 'C', 'Ca', 'Co', 'Cr', 'F', 'Fe', 'He', 'K', 'Mg', 'Mn',
       'N', 'Na', 'Ne', 'Ni', 'O', 'P', 'S', 'Si', 'Ti', 'V'], 
      dtype='<U2')